In [12]:
import tensorflow as tf
import numpy as np

(X_train_full, y_train_full), (X_test, y_test) =  tf.keras.datasets.mnist.load_data()

X_train_full = X_train_full / 255.
X_test = X_test / 255.
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [14]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=3, padding="same",
                           activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Conv2D(64, kernel_size=3, padding="same",
                           activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])



In [15]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

Epoch 1/10
1719/1719 [==============================] - 263s 151ms/step - loss: 0.1896 - accuracy: 0.9419 - val_loss: 0.0450 - val_accuracy: 0.9870
Epoch 2/10
1719/1719 [==============================] - 288s 168ms/step - loss: 0.0812 - accuracy: 0.9759 - val_loss: 0.0432 - val_accuracy: 0.9886
Epoch 3/10
1719/1719 [==============================] - 305s 177ms/step - loss: 0.0609 - accuracy: 0.9815 - val_loss: 0.0396 - val_accuracy: 0.9894
Epoch 4/10
1719/1719 [==============================] - 221s 128ms/step - loss: 0.0482 - accuracy: 0.9851 - val_loss: 0.0349 - val_accuracy: 0.9914
Epoch 5/10
1719/1719 [==============================] - 222s 129ms/step - loss: 0.0415 - accuracy: 0.9871 - val_loss: 0.0315 - val_accuracy: 0.9922
Epoch 6/10
1719/1719 [==============================] - 223s 130ms/step - loss: 0.0327 - accuracy: 0.9900 - val_loss: 0.0411 - val_accuracy: 0.9906
Epoch 7/10
1719/1719 [==============================] - 218s 127ms/step - loss: 0.0304 - accuracy: 0.9899 - val_

[0.030557546764612198, 0.9921000003814697]

In [26]:
import tensorflow_datasets as tfds

test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [28]:
batch_size = 32
preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])

train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))
train_set = train_set.shuffle(1000, seed=42).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)
test_set = test_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

In [30]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                                     include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(5, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [31]:
for layer in base_model.layers:
    layer.trainable = False

In [32]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=3)

Epoch 1/3
13/86 [===>..........................] - ETA: 6:00 - loss: 0.8762 - accuracy: 0.6562

In [ ]:
for layer in base_model.layers[56:]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=10)